# Swissimage YOLOv8 Segmentation – Minimal lauffaehiges Notebook

## 1) Installation, Drive-Mount, Imports, Seeds, GPU-Check

In [ ]:
# -- Pakete --
!pip -q install -U pip setuptools wheel

# Torch passend zur Umgebung (GPU/CPU)
try:
    import torch
    HAS_CUDA = torch.cuda.is_available()
except Exception:
    HAS_CUDA = False

if HAS_CUDA:
    # CUDA 12.x Build (Colab Standard)
    !pip -q install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
else:
    # CPU Build
    !pip -q install --upgrade torch torchvision==0.23 torchaudio --index-url https://download.pytorch.org/whl/cpu

# Geopandas-Stack + Ultralytics
!pip -q install ultralytics geopandas shapely rasterio pyproj rtree tqdm opencv-python-headless

# -- Drive mount --
from google.colab import drive
try:
    drive.mount('/content/drive', force_remount=True)
except Exception:
    drive._mount('/content/drive')

# -- Imports & Seeds --
import warnings, os, random, shutil, json
from pathlib import Path
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon, box
from shapely.ops import unary_union
import rasterio
from rasterio.transform import rowcol, xy
from PIL import Image
from tqdm import tqdm
from ultralytics import YOLO
import torch

def set_seeds(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
set_seeds(42)

print("CUDA verfuegbar:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
DEVICE = 0 if torch.cuda.is_available() else 'cpu'

Mounted at /content/drive
CUDA verfuegbar: False


## 2) Pfade setzen und Daten pruefen
Passe **nur** die beiden Variablen `IMAGES_DIR` und `GPKG_FILE` an, falls deine Dateien woanders liegen.

In [ ]:
# -- Pfade (anpassen, falls noetig) --
IMAGES_DIR = Path('/content/drive/MyDrive/swissimage_tiles')  # Ordner mit deinen GeoTIFFs
GPKG_FILE  = Path('/content/drive/MyDrive/BBox_Fussgaengerstreifen_Stadt_Bern.gpkg')  # dein GPKG

BASE       = Path('/content/drive/MyDrive/Swissimage_Segmentation')
YOLO_DIR   = BASE/'data'/'yolo'
IMG_TRN    = YOLO_DIR/'images'/'train'
IMG_VAL    = YOLO_DIR/'images'/'val'
LBL_TRN    = YOLO_DIR/'labels'/'train'
LBL_VAL    = YOLO_DIR/'labels'/'val'
for p in [IMG_TRN, IMG_VAL, LBL_TRN, LBL_VAL, BASE/'runs', BASE/'outputs']:
    p.mkdir(parents=True, exist_ok=True)

# TIFF-Liste robust
def list_tiffs(d: Path):
    pats = ('*.tif','*.tiff','*.TIF','*.TIFF')
    files = []
    for pat in pats: files += list(d.glob(pat))
    return sorted(files)

tiffs = list_tiffs(IMAGES_DIR)
print(f"Gefundene Raster in {IMAGES_DIR}: {len(tiffs)}")
if tiffs[:5]:
    for p in tiffs[:5]:
        print("  -", p.name)

print("GPKG existiert:", GPKG_FILE.exists(), "|", GPKG_FILE)
if not tiffs:
    raise FileNotFoundError(f"Keine TIF/TIFFs in {IMAGES_DIR} gefunden.")
if not GPKG_FILE.exists():
    raise FileNotFoundError(f"GPKG fehlt: {GPKG_FILE}")


Gefundene Raster in /content/drive/MyDrive/swissimage_tiles: 83
  - swissimage-dop10_2024_2589-1196_0.1_2056.tif
  - swissimage-dop10_2024_2589-1197_0.1_2056.tif
  - swissimage-dop10_2024_2589-1198_0.1_2056.tif
  - swissimage-dop10_2024_2590-1196_0.1_2056.tif
  - swissimage-dop10_2024_2590-1197_0.1_2056.tif
GPKG existiert: True | /content/drive/MyDrive/BBox_Fussgaengerstreifen_Stadt_Bern.gpkg


## 3) Labels aus GPKG erzeugen (YOLOv8-Seg), Dataset anlegen, Split

In [ ]:
# Parameter
CLASSES = ['crosswalk']
VAL_FRACTION = 0.2
MIN_AREA_M2 = 2.0  # zu kleine Polygone verwerfen

# Vektordaten laden und pruefen
gdf = gpd.read_file(GPKG_FILE)
print("GPKG-CRS:", gdf.crs)
if str(gdf.crs) not in ("EPSG:2056","2056"):
    gdf = gdf.to_crs(2056)

# Filter: gueltige Geometrien, Mindestflaeche
gdf = gdf[gdf.geometry.notnull()].copy()
gdf = gdf[gdf.is_valid].copy()
try:
    gdf['area'] = gdf.geometry.area
    gdf = gdf[gdf['area'] >= MIN_AREA_M2].copy()
except Exception:
    pass

# Polygon -> YOLO-Seg-Zeile
def polygon_to_yolo_line(poly, w, h, class_id: int):
    if poly.is_empty:
        return None
    xys = list(poly.exterior.coords)
    pts = []
    for xpix, ypix in xys:
        xn = max(0.0, min(1.0, xpix / float(w)))
        yn = max(0.0, min(1.0, ypix / float(h)))
        pts += [f"{xn:.6f}", f"{yn:.6f}"]
    if len(pts) < 12:  # mind. 6 Punkte
        return None
    return str(class_id) + " " + " ".join(pts)

# Split (kachelnweise, reproduzierbar)
rng = np.random.RandomState(42)
idx = np.arange(len(tiffs))
rng.shuffle(idx)
cut = int(len(idx) * (1.0 - VAL_FRACTION))
train_idx = set(idx[:cut])

def write_label_and_copy(image_path: Path, is_train: bool):
    out_img = (IMG_TRN if is_train else IMG_VAL)/image_path.name
    out_lbl = (LBL_TRN if is_train else LBL_VAL)/(image_path.stem + '.txt')

    with rasterio.open(image_path) as src:
        if str(src.crs) not in ("EPSG:2056","2056"):
            raise ValueError(f"Raster {image_path.name} hat CRS {src.crs}, erwartet EPSG:2056.")
        transform = src.transform
        width, height = src.width, src.height
        # Bildausdehnung in Kartenkoordinaten
        left, bottom = xy(transform, height, 0)
        right, top   = xy(transform, 0, width)
        bbox_world = box(min(left,right), min(bottom,top), max(left,right), max(bottom,top))

        sub = gdf[gdf.geometry.intersects(bbox_world)].copy()
        if sub.empty:
            out_lbl.write_text("")
            shutil.copy2(image_path, out_img)
            return

        lines = []
        for geom in sub.geometry:
            if geom.is_empty:
                continue
            geom_clipped = geom.intersection(bbox_world)
            if geom_clipped.is_empty:
                continue

            def world_to_pixel_coords(geom_world):
                exterior = []
                for xw, yw in np.array(geom_world.exterior.coords):
                    r, c = rowcol(transform, xw, yw)
                    exterior.append((float(c), float(r)))  # (x,y) = (col,row)
                return Polygon(exterior)

            if isinstance(geom_clipped, MultiPolygon):
                for part in geom_clipped.geoms:
                    poly_px = world_to_pixel_coords(part)
                    line = polygon_to_yolo_line(poly_px, width, height, class_id=0)
                    if line: lines.append(line)
            elif isinstance(geom_clipped, Polygon):
                poly_px = world_to_pixel_coords(geom_clipped)
                line = polygon_to_yolo_line(poly_px, width, height, class_id=0)
                if line: lines.append(line)

        out_lbl.write_text("\n".join(lines))
        shutil.copy2(image_path, out_img)

# Ausgabeordner leeren
for p in [IMG_TRN, IMG_VAL, LBL_TRN, LBL_VAL]:
    for f in p.glob('*'):
        try: f.unlink()
        except IsADirectoryError: shutil.rmtree(f, ignore_errors=True)

# Erzeugen
for i, tif in enumerate(tqdm(tiffs, desc="Erzeuge YOLO-Labels")):
    write_label_and_copy(tif, is_train=(i in train_idx))

# dataset.yaml schreiben
dataset_yaml = f"""path: {YOLO_DIR}
train: images/train
val: images/val
names: {CLASSES}
"""
(YOLO_DIR/'dataset.yaml').write_text(dataset_yaml)

print("Fertig: YOLO-Dataset unter", YOLO_DIR)
print("Train images:", len(list(IMG_TRN.glob('*.tif'))) + len(list(IMG_TRN.glob('*.tiff'))))
print("Val images  :", len(list(IMG_VAL.glob('*.tif'))) + len(list(IMG_VAL.glob('*.tiff'))))

GPKG-CRS: EPSG:2056


Erzeuge YOLO-Labels:   1%|          | 1/83 [00:01<01:52,  1.37s/it]

## 4) Training starten (YOLOv8-Seg)

In [ ]:
EPOCHS = 5
BATCH  = 8
IMGSZ  = 1024
WEIGHTS = 'yolov8s-seg.pt'  # oder 'yolov8n-seg.pt'

model = YOLO(WEIGHTS)
results = model.train(
    data=str(YOLO_DIR/'dataset.yaml'),
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH,
    device=DEVICE,
    project=str(BASE/'runs'),
    name='yolov8_crosswalk_seg',
    seed=42,
    patience=20
)
print("Training abgeschlossen. Artefakte unter:", results.save_dir)

## 5) Optional: Schnelle Inferenz auf dem Validierungsset

In [ ]:
best = Path(results.save_dir)/'weights'/'best.pt'
model = YOLO(str(best))
pred_dir = BASE/'outputs'/'pred_val'
pred_dir.mkdir(parents=True, exist_ok=True)

r = model.predict(
    source=str(IMG_VAL),
    project=str(pred_dir),
    name='seg',
    save=True,
    save_txt=False,
    conf=0.25,
    iou=0.5,
    device=DEVICE
)
print("Predictions gespeichert unter:", pred_dir)